# Low Level Widget Tutorial

## How do they fit into the picture?

One of the goals of the Jupyter Notebook is to minimize the “distance” the user is from their data.  This means allowing the user to quickly view and manipulate the data.  

| ![inputoutput](images/inputoutput.PNG)                                                                           | ![example-widgets](images/widgets.PNG)                                                                                                                      |
|-------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------|
| Before the widgets, this was just the segmentation of code and results from executing those segments. | Widgets further decrease the distance between the user and their data by allowing UI interactions to directly manipulate data in the kernel. |

## How?

Jupyter interactive widgets are interactive elements, think sliders, text boxes, buttons, that have representations both in the kernel (place where code is executed) and the front-end (the Notebook web interface).  To do this, a clean, well-abstracted communication layer must exist.

## Comms

This is where Jupyter notebook “comms” come into play.  The comm API is a symmetric, asynchronous, fire and forget style messaging API.  It allows the programmer to send JSON-able blobs between the front-end and the back-end.  The comm API hides the complexity of the webserver, ZMQ, and websockets.

![comms](images/transport.svg)

## Synchronized state

Using comms, the widget base layer is designed to keep state in sync.  In the kernel, a Widget instance exists.  This Widget instance has a corresponding WidgetModel instance in the front-end.  The Widget and WidgetModel store the same state.  The widget framework ensures both models are kept in sync with each other.  If the WidgetModel is changed in the front-end, the Widget receives the same change in the kernel.  Vice versa, if the Widget in the kernel is changed, the WidgetModel in the front-end receives the same change.  There is no single source of truth, both models have the same precedence.  Although a notebook has the notion of cells, neither Widget or WidgetModel are bound to any single cell.

![synchronized state](images/state_sync.svg)

## Models and Views

In order for the user to interact with widgets on a cell by cell basis, the WidgetModels are represented by WidgetViews.  Any single WidgetView is bound to a single cell.  Multiple WidgetViews can be linked to a single WidgetModel.  This is how you can redisplay the same Widget multiple times and it still works.  To accomplish this, the widget framework uses Backbone.js.  In a traditional MVC framework, the WidgetModel is the (M)odel, and the WidgetView is both the (V)iew and (C)ontroller. Meaning that, the views both display the state of the model and manipulate it.  Think about a slider control, it both displays the value and allows the user to change the value by dragging the slide handle.

In [ ]:
from ipywidgets import *
from IPython.display import display
w = IntSlider()
display(w, w)

In [ ]:
display(w)

![model-view venn diagram](images/assoc.svg)

## Code execution

The user code required to display a simple FloatSlider widget is:

```python
from ipywidgets import FloatSlider
from IPython.display import display
slider = FloatSlider()
display(slider)
```

In order to understand how a widget is displayed, one must understand how code is executed in the Notebook.  Execution begins in the code cell.  A user event triggers the code cell to send an evaluate code message to the kernel, containing all of the code in the code cell.  This message is given a GUID, which the front-end associates to the code cell, and remembers it (**important**).

![execution-1](images/execute.svg)

Once that message is received by the kernel, the kernel immediately sends the front-end an “I’m busy” status message.  The kernel then proceeds to execute the code.

![execution-2](images/busy.svg)

## Model construction

When a Widget is constructed in the kernel, the first thing that happens is that a comm is constructed and associated with the widget.  When the comm is constructed, it is given a GUID (globally unique identifier).  A comm-open message is sent to the front-end, with metadata stating that the comm is a widget comm and what the corresponding WidgetModel class is.  

![model-construction](images/widgetcomm.svg)

The WidgetModel class is specified by module and name.  Require.js is then used to asynchronously load the WidgetModel class. The message triggers a comm to be created in the front-end with the same GUID as the back-end.  Then, the new comm gets passed into the WidgetManager in the front-end, which creates an instance of the WidgetModel class, linked to the comm.  Both the Widget and WidgetModel repurpose the comm GUID as their own.

![construction-2](images/widgetcomm2.svg)

Asynchronously, the kernel sends an initial state push, containing the initial state of the Widget, to the front-end, immediately after the comm-open message.  This state message may or may not be received by the time the WidgetModel is constructed.  Regardless, the message is cached and gets processed once the WidgetModel has been constructed.  The initial state push is what causes the WidgetModel in the front-end to become in sync with the Widget in the kernel.

![construction-3](images/state.svg)

## Displaying a view

After the Widget has been constructed, it can be displayed.  Calling `display(widgetinstance)` causes a specially named repr method in the widget to run.  This method sends a message to the front-end that tells the front-end to construct and display a widget view.  The message is in response to the original code execution message, and the original message’s GUID is stored in the new message’s header.  When the front-end receives the message, it uses the original message’s GUID to determine what cell the new view should belong to.  Then, the view is created, using the WidgetView class specified in the WidgetModel’s state.  The same require.js method is used to load the view class.  Once the class is loaded, an instance of it is constructed, displayed in the right cell, and registers listeners for changes of the model.

![display a view](images/display.svg)

## Widget skeleton

In [ ]:
%%javascript
this.model.get('count');
this.model.set('count', 999);
this.touch();

/////////////////////////////////

this.colorpicker = document.createElement('input');
this.colorpicker.setAttribute('type', 'color');
this.el.appendChild(this.colorpicker);

Since widgets exist in both the front-end and kernel, they consist of both Python (if the kernel is IPython) and Javascript code.  A boilerplate widget can be seen below:

Python:
```python
from ipywidgets import DOMWidget
from traitlets import Unicode, Int
 
class MyWidget(DOMWidget):
	_view_module = Unicode('mywidget').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
	_view_name = Unicode('MyWidgetView').tag(sync=True)
	count = Int().tag(sync=True)
```

JavaScript:
```js
define('mywidget', ['@jupyter-widgets/base'], function(widgets) {
	var MyWidgetView = widgets.DOMWidgetView.extend({
    	render: function() {
        	MyWidgetView.__super__.render.apply(this, arguments);
        	this._count_changed();
        	this.listenTo(this.model, 'change:count', this._count_changed, this);
    	},
 
    	_count_changed: function() {
        	var old_value = this.model.previous('count');
        	var new_value = this.model.get('count');
        	this.el.textContent = String(old_value) + ' -> ' + String(new_value);
    	}
	});
 
	return {
    	MyWidgetView: MyWidgetView
	}
});
```

Describing the Python:  

The base widget classes are `DOMWidget` and `Widget`. The `DOMWidget` class represents a widget that is represented in the page as an HTML DOM element. The `Widget` class is more general and can be used for objects that may not live on the page as a DOM element (for example, a widget inheriting from `Widget` may represent a Javascript object).

`_view_module`, `_view_module_version`, and `_view_name` are how the front-end knows what view class to construct for the model.

`sync=True` is what makes the traitlets behave like state.

A similarly named `_model_module`, `_model_module_version`, and `_model_name` can be used to specify the corresponding WidgetModel.

`count` is an example of a custom piece of state.

Describing the JavaScript:  

The `define` call asynchronously loads the specified dependencies, and then passes them in as arguments into the callback.  Here, the only dependency that is loaded is the base widget module.

Custom views inherit from either `DOMWidgetView` or `WidgetView`. The `DOMWidgetView` class is for widgets that render themselves into a DOM element, and the `WidgetView` class does not make this assumption.

Custom models inherit from `WidgetModel`.

The `render` method is what is called to render the view’s contents.  If the view is a `DOMWidgetView`, the `.el` attribute contains the DOM element that will be displayed on the page.

`.listenTo` allows the view to listen to properties of the model for changes.

`_count_changed` is an example of a method that could be used to handle model changes.

`this.model` is how the corresponding model can be accessed.

`this.model.previous` will get the previous value of the trait.

`this.model.get` will get the current value of the trait.

`this.model.set` followed by `this.model.save_changes();` changes the model.  
Use the view method `touch` instead of `model.save_changes` to associate the changes with the current view, thus associating any response messages with the view’s cell.

The dictionary returned is the public members of the module.


## Serialization of widget attributes

Widget trait attributes tagged with `sync=True` are synchronized with the JavaScript model instance on the JavaScript side. For this reason, they need to be serialized into `json`.

By default, basic Python types such as `int`, `float`, `list` and `dict` are simply be mapped to `Number`, `Array` and `Object`. For more complex types, serializers and de-serializers must be specified on both the Python side and the JavaScript side.


### Custom serialization and de-serialization on the Python side

In many cases, a custom serialization must be specified for trait attributes. For example,

 - if the trait attribute is not json serializable
 - if the trait attribute contains data that is not needed by the JavaScript side.
 
Custom serialization can be specified for a given trait attribute through the `to_json` and `from_json` metadata. These must be functions that take two arguments

 - the value to be [de]serialized
 - the instance of the underlying widget model.
 
In most cases, the second argument is not used in the implementation of the serializer. 

**Example**

For example, in the case of the `value` attribute of the `DatePicker` widget, the declaration is

```python
value = Datetime(None, allow_none=True).tag(sync=True, to_json=datetime_to_json, from_json=datetime_from_json)
```

where `datetime_to_json(value, widget)` and `datetime_from_json(value, widget)` return or handle json data-structures that are amenable to the front-end.

**The case of parent-child relationships between widget models**

When a widget model holds other widget models, you must use the serializers and deserializers provided in ipywidgets packed into the `widget_serialization` dictionary.

For example, the `HBox` widget declares its `children` attribute in the following fashion:

```python
from .widget import widget_serialization

[...]

children = Tuple().tag(sync=True, **widget_serialization)
```

The actual result of the serialization of a widget model is a string holding the widget id prefixed with `"IPY_MODEL_"`.

### Custom serialization and de-serialization on the JavaScript side

In order to mirror the custom serializer and deserializer of the Python side, symmetric methods must be provided on the JavaScript side.

On the JavaScript side, serializers are specified through the `serializers` class-level attribute of the widget model.

They are generally specified in the following fashion, extending the dictionary of serializers and serializers of the base class. In the following example, which comes from the `DatePicker`, the deserializer for the `value` attribute is specified.

```JavaScript
static serializers = _.extend({
    value: {
        serialize: serialize_datetime,
        deserialize: deserialize_datetime
    }
}, BaseModel.serializers)
```

Custom serializers are functions taking two arguments: the value of the object to [de]serialize, and the widget manager. In most cases, the widget manager is actually not used.

## Installation

Because the API of any given widget **must exist in the kernel**, the kernel is the natural place for widgets to be installed.  However, **kernels, as of now, don’t host static assets**.  Instead, static assets are hosted by the webserver, which is the entity that sits between the kernel and the front-end.  This is a problem because it means widgets have components that need to be **installed both in the webserver and the kernel**.  The kernel components are easy to install, because you can rely on the language’s built-in tools.  The static assets for the webserver complicate things, because an extra step is required to let the webserver know where the assets are.

## Static assets

In the case of the classic Jupyter notebook, static assets are made available to the Jupyter notebook in the form of a Jupyter extension. JavaScript bundles are copied in a directory accessible through the `nbextensions/` handler.  Nbextensions also have a mechanism for running your code on page load.  This can be set using the install-nbextension command.

## Distribution

Two template projects are available in the form of cookiecutters:

- JavaScript: https://github.com/jupyter-widgets/widget-cookiecutter
- TypeScript: https://github.com/jupyter-widgets/widget-ts-cookiecutter

These cookiecutters are meant to help custom widget authors get started with the packaging and the distribution of Jupyter interactive widgets.

They produce a project for a widget library following the current best practices for using interactive widgets. An implementation for a placeholder "Hello World" widget is provided.